In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import sys
import os

In [3]:
from models import *

/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [4]:
from fastai.column_data import *

In [5]:
from fastai.conv_learner import *
from fastai.dataset import *

In [6]:
import numpy as np

In [ ]:
ls

In [ ]:
ls KITTI/2011_10_03/2011_10_03_drive_0034_sync/image_03/data/0000003783.png

In [7]:
class UnsupFilesDataset(FilesDataset):
    def __init__(self, fnames, y, transform, path):
        super().__init__(fnames, transform, path)
    def get_y(self, i): return None
    def get_c(self): return 0

In [8]:
train, val= pd.read_csv('./KITTI/training.csv'), pd.read_csv('./KITTI/validation.csv')

In [9]:
trn0, trn1, trn2, trn_camera = train.t0, train.t1,train.t2, train[['fx', 'fy', 'cx', 'cy']]

In [10]:
val0, val1, val2, val_camera  = val.t0, val.t1, val.t2, val[['fx', 'fy', 'cx', 'cy']]

In [11]:
trn_camera.head()

,fx,fy,cx,cy
0,718.3351,718.3351,600.3891,181.5122
1,718.3351,718.3351,600.3891,181.5122
2,718.3351,718.3351,600.3891,181.5122
3,718.3351,718.3351,600.3891,181.5122
4,718.3351,718.3351,600.3891,181.5122


val_idxs = get_cv_idxs(len(fnames), val_pct=min(0.01/keep_pct, 0.1))
((val_x,trn_x),(val_y,trn_y)) = split_by_idx(val_idxs, np.array(fnames), np.array(fnames))
len(val_x),len(trn_x)

In [12]:
aug_tfms = [
    #RandomDihedral(tfm_y=TfmType.NO),
    RandomLighting(b=0.05, c=0.05)]

In [13]:
arch = f
scale,bs = 1, 4
sz = 256 #np.array([345, 1242]) 
sz_y = sz // scale 

# scale,bs = 4,32

PATH = 'Fastai_TRN'

In [14]:
class UnShuffleImageData(ImageData):
    def __init__(self, path, datasets, bs, num_workers, classes):
        trn_ds,val_ds,fix_ds,aug_ds,test_ds,test_aug_ds = datasets
        self.path,self.bs,self.num_workers,self.classes = path,bs,num_workers,classes
        self.trn_dl,self.val_dl,self.fix_dl,self.aug_dl,self.test_dl,self.test_aug_dl = [
            self.get_dl(ds,shuf) for ds,shuf in [
                (trn_ds,False),(val_ds,False),(fix_ds,False),(aug_ds,False),
                (test_ds,False),(test_aug_ds,False)
            ]
        ]

In [15]:
def get_MD(trn, val):
    tfms = tfms_from_model(
        arch,
        sz,
        crop_type=CropType.NO,
        tfm_y=TfmType.NO,
        aug_tfms=aug_tfms,
        sz_y=sz_y)
    datasets = ImageData.get_ds(
        UnsupFilesDataset,
        (trn, None),
        (val, None),
        tfms, path='KITTI/')
    md = UnShuffleImageData(
        PATH,
        datasets,
        bs, num_workers=16,
        classes=None)
    
    return md

In [16]:
def get_cam(trn_cam, val_cam):
    return ColumnarModelData.from_data_frames(
        path= None,
        trn_df=trn_cam, val_df=val_cam,
        trn_y=None, val_y=None,
        cat_flds=[],
        bs=bs, shuffle=False)

In [17]:
MD1 = get_MD(trn0, val0)
MD2 = get_MD(trn1, val1)
MD3 = get_MD(trn2, val2)
MDcam = get_cam(trn_camera, val_camera)

In [18]:
class Loss(nn.Module):
    def __init__(self, scale=0.01, Tscale=0.01):
        super().__init__()
        self.appr = TriAppearanceLoss()
        self.smooth = SmoothLoss()
        self.scale = scale
    def forward(self, d1, d2 ,d3, poses_x2, x1, x2, x3, camera):
        return self.appr(d1, d3, poses_x2, x1, x2, x3, camera)+scale * self.smooth(d1, d2, d3)
        

In [19]:
m = TriDepth(get_base(), 1).cuda()

In [20]:
l = Loss().cuda() 

In [21]:
opt = optim.Adam(m.parameters())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
imgbatch1 = next(iter(MD1.trn_dl))
imgbatch2 = next(iter(MD2.trn_dl))
imgbatch3 = next(iter(MD3.trn_dl))
cambatch = next(iter(MDcam.trn_dl))

img1 = MD1.trn_ds.denorm(imgbatch1)[0]
img2 = MD2.trn_ds.denorm(imgbatch2)[0]
img3 = MD3.trn_ds.denorm(imgbatch3)[0]
cam = cambatch[1][0]

f, axs = plt.subplots(1, 3)
axs[0].imshow(img1)
axs[1].imshow(img2)
axs[2].imshow(img3)

print(cam.cpu().numpy().reshape(-1))

In [ ]:
oo= torch.zeros([1,1]).type(torch.FloatTensor).cuda()
Eeye=torch.eye(3).unsqueeze(0).type(torch.FloatTensor).cuda()
def factorize(vecs, dim):
    mags = vecs.norm(p=2, dim=dim, keepdim=True)
    return vecs/mags, mags

def rot_vec2mat(rot_vecs):
    batch, _ = rot_vecs.size()
    directs, angles = factorize(rot_vecs, 1)

    K0 = directs[:,:1]
    K1 = directs[:,1:2]
    K2 = directs[:,2:]

    o = oo.repeat(batch, 1)
    eye = Eeye.repeat(batch, 1, 1)

    # print(K0.type, K2.type, K1.type, o.type, eye.type)

    K = torch.cat(seq=(o, -K2, K1, K2, o, K0, -K1, K0, o), dim=1).view(-1, 3, 3)
    
    #pdb.set_trace()
    angles = angles.unsqueeze(-1)
    res1 = K * angles.sin()
    res2 = torch.bmm(K,K) * (1-angles.cos())
    print(res1.size(), res2.size(), eye.size())
    return eye + res1 + res2

In [ ]:
l.appr.offset.o.type()

In [ ]:
xy = torch.meshgrid([torch.arange(10), torch.arange(20)])

In [ ]:
offset1 = l.appr.offset.forward( poses_x2[:,0], d1 )

In [25]:
DL1, DL2, DL3, DLcam = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl), iter(MDcam.trn_dl)
imgs1, imgs2, imgs3, cam = V([next(DL1), next(DL2),next(DL3), next(DLcam)[1]])

In [26]:
opt.zero_grad()

d1, d2, d3, poses_x2 = m(imgs1, imgs2, imgs3)
loss = l(d1, d2, d3, poses_x2, imgs1, imgs2, imgs3, cam)
loss.backward()
opt.step()

RuntimeError: The size of tensor a (256) must match the size of tensor b (4) at non-singleton dimension 1

In [ ]:
def fit(model, metric, opt, MD1, MD2, MD3):
    DL1, DL2, DL3 = iter(MD1.trn_dl), iter(MD2.trn_dl), iter(MD3.trn_dl)
    losses = []
    for i in tqdm(range(len(MD1.trn_ds))):
        opt.zero_grad()
        imgs1, imgs2, imgs3= V([next(DL1), next(DL2),next(DL3)])
        
        d1, d2, d3, poses_x2 = model(imgs1, imgs2, imgs3)
        loss = metric(d1, d2, d3, poses_x2, imgs1, imgs2, imgs3)
        loss.backward()
        opt.step()
        losses.append(loss)
    return torch.cat(loesses).mean()

In [ ]:
a=torch.randn(1,2,6); a

In [ ]:
b=a[:,0]

In [ ]:
b[:,:3]

In [ ]:
fit(m, l, opt, MD1, MD2, MD3)